In [24]:
import tensorflow as tf
import socketio
import matplotlib.pyplot as plt
from time import sleep
import requests
import urllib

In [25]:
BASE_URL = 'http://localhost:3000'

In [67]:
def plot_image(image):
    plt.figure(figsize=(9, 6))
    plt.imshow(image.numpy().astype("uint8"))
    plt.axis('off')
    plt.show()

In [79]:
class SimulationNamespace(socketio.ClientNamespace):
    # def on_connect(self):
    #     print('Connected')
    
    # def on_disconnect(self):
    #     print('Disconnected')

    def on_connect_error(self, error):
        print(error)

    def on_render(self, data_uri):
        res = urllib.request.urlopen(data_uri)
        image = tf.io.decode_png(res.file.read(), channels=4)
        # plot_image(image)
        actions = get_actions(image)
        send_actions(actions, )
    
    def on_message(self, message):
        print(message)

In [85]:
def get_actions(image):
    random_vec = tf.random.uniform((2, 1), -20, 20, dtype=tf.int32)
    return random_vec

def send_actions(actions, socket, namespace):
    x, y = tf.reshape(actions, [-1]).numpy()
    socket.emit('sim:move', data={'x': int(x), 'y': int(y)}, namespace=namespace)


In [89]:
response = requests.get(BASE_URL+'/sims')
id_list = response.json()

sim_id = id_list[0] 
namespace = f'/sim-{sim_id}'

socket = socketio.Client(reconnection_attempts=3)
socket.register_namespace(SimulationNamespace(namespace))

socket.connect(BASE_URL)
# socket.emit('sim:render', namespace=namespace)
while True:
    try:
        actions = get_actions(None)
        send_actions(actions, socket, namespace)
        socket.sleep(.1)
    except KeyboardInterrupt:
        break

socket.disconnect()
socket.wait()